In [1]:
# 进入/home/aistudio/work目录
%cd /home/aistudio/work

/home/aistudio/work


In [ ]:
# 克隆paddleseg
# !git clone -b v2.3.0 https://gitee.com/paddlepaddle/PaddleSeg.git


In [ ]:
# 安装paddleseg
!pip install paddleseg

In [3]:
# 进入/home/aistudio/work/PaddleSeg目录
%cd PaddleSeg

/home/aistudio/work/PaddleSeg


In [13]:
# 解压训练数据到/home/aistudio/work/data下
!unzip -q /home/aistudio/data/data77571/train_and_label.zip -d data/

In [14]:
# 解压测试集数据到/home/aistudio/work/data下
!unzip -q /home/aistudio/data/data77571/img_test.zip -d data/

In [ ]:
# 划分训练集、测试集，比例为8：2

import os
import random
import shutil
from tqdm import tqdm

label_path = "data/lab_train"
img_path = "data/img_train"
save_path = "data/dataset"

train_img_path = os.path.join(save_path, "train", "images")
train_ann_path = os.path.join(save_path, "train", "annotations")

val_img_path = os.path.join(save_path, "val", "images")
val_ann_path = os.path.join(save_path, "val", "annotations")

os.makedirs(train_img_path)
os.makedirs(train_ann_path)
os.makedirs(val_img_path)
os.makedirs(val_ann_path)

name_list = os.listdir(img_path)
# print(name_list)
random.shuffle(name_list)

train_list_path = os.path.join(save_path, "train.txt")
val_list_path = os.path.join(save_path, "val.txt")
train_file = open(train_list_path, "a")
val_file = open(val_list_path, "a")

for name in tqdm(name_list[:int(len(name_list)*0.8)]):
    img_old_path = os.path.join(img_path, name)
    img_new_path = os.path.join(train_img_path, name)
    shutil.copy(img_old_path, img_new_path)

    ann_old_path = os.path.join(label_path, name.replace("jpg", "png"))
    ann_new_path = os.path.join(train_ann_path, name.replace("jpg", "png"))
    shutil.copy(ann_old_path, ann_new_path)
    
    train_file.write("train/images/{} train/annotations/{}\n".format(name, name.replace("jpg", "png")))

for name in tqdm(name_list[int(len(name_list)*0.8):]):
    img_old_path = os.path.join(img_path, name)
    img_new_path = os.path.join(val_img_path, name)
    shutil.copy(img_old_path, img_new_path)

    ann_old_path = os.path.join(label_path, name.replace("jpg", "png"))
    ann_new_path = os.path.join(val_ann_path, name.replace("jpg", "png"))
    shutil.copy(ann_old_path, ann_new_path)

    val_file.write("val/images/{} val/annotations/{}\n".format(name, name.replace("jpg", "png")))

In [16]:
# 数据读取配置文件
!cp /home/aistudio/config/custom.yml configs/_base_/custom.yml

In [17]:
# ocrnet配置文件
!cp /home/aistudio/config/ocrnet.yml configs/ocrnet/ocrnet.yml

In [18]:
# segformer_B3配置文件
!cp /home/aistudio/config/segformer.yml configs/segformer/segformer.yml

In [ ]:
# 模型训练

# ocrnet
# !python train.py --config configs/ocrnet/ocrnet.yml --do_eval --use_vdl --save_interval 500 --save_dir output

# segformer
!python train.py --config configs/segformer/segformer.yml --log_iters 2200 --use_vdl --save_interval 2200 --save_dir output
# !python train.py --config configs/segformer/segformer.yml --log_iters 1 --use_vdl --save_interval 1 --save_dir output


In [ ]:
# !cp -r output/ /home/aistudio/output

In [ ]:
# 模型测试
!python val.py --config configs/segformer/segformer.yml --model_path=output/iter_16000/model.pdparams


In [4]:
# 推理预测
!python predict.py --config configs/segformer/segformer.yml --model_path output/iter_94600/model.pdparams --image_path data/img_testA --save_dir output/result

2021-11-01 09:30:10 [INFO]	
---------------Config Information---------------
batch_size: 24
iters: 160000
loss:
  coef:
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 6.0e-05
  power: 1
  type: PolynomialDecay
model:
  num_classes: 5
  pretrained: https://bj.bcebos.com/paddleseg/dygraph/mix_vision_transformer_b3.tar.gz
  type: SegFormer_B3
optimizer:
  beta1: 0.9
  beta2: 0.999
  type: AdamW
  weight_decay: 0.01
train_dataset:
  dataset_root: data/dataset
  mode: train
  num_classes: 5
  train_path: data/dataset/train.txt
  transforms:
  - max_scale_factor: 2.0
    min_scale_factor: 0.5
    scale_step_size: 0.25
    type: ResizeStepScaling
  - crop_size:
    - 256
    - 256
    type: RandomPaddingCrop
  - type: RandomHorizontalFlip
  - brightness_range: 0.4
    contrast_range: 0.4
    saturation_range: 0.4
    type: RandomDistort
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: data/dataset
  mode: val
  num_classes: 5
  transforms:

In [5]:
%cd output/result

/home/aistudio/work/PaddleSeg/output/result


In [6]:
!zip -q -r result.zip pseudo_color_prediction

In [ ]:
%cd ..